# Projeto de Web Scraping

### Ainda não tenho muita ideia do que fazer com esse código... ele entra no site da investing.com na área de bancos centrais e pega alguns dados

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from os import getcwd
from datetime import datetime

In [15]:
def formatar_data(agora: datetime.now()):
    hora = agora.hour
    minuto = agora.minute
    segundo = agora.second
    if hora < 10:
        hora = f'0{hora}'
    if minuto < 10:
        minuto = f'0{minuto}'
    if segundo < 10:
        segundo = f'0{segundo}'
    return f'{agora.day}-{agora.month}-{agora.year} - {hora};{minuto};{segundo}'

In [16]:
# abrir navegador
navegador = webdriver.Chrome()
# entrar no site
navegador.get('https://br.investing.com/central-banks/')

In [17]:
def formatar(banco):
    return float(banco['Taxa Atual'].replace(',', '.').replace('%', ''))

In [18]:
# pegar os dados
bancos_centrais = [
    # {'Banco Central': ..., 'Taxa Atual': x%, 'Reunião': xx.xx.xxxx, 'Última Mudança': xx.xx.xxxx (xxbp)}
]
for i in range(1, 12):
    banco_central = navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[2]/a').text
    banco_central += ' ' + navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[2]/span').text
    taxa_atual = navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[3]').text
    reuniao = navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[4]').text
    ultima_mudanca = navegador.find_element(By.XPATH, f'//*[@id="curr_table"]/tbody/tr[{i}]/td[5]').text
    bancos_centrais.append({
        'Banco Central': banco_central,
        'Taxa Atual': taxa_atual,
        'Reunião': reuniao,
        'Última Mudança': ultima_mudanca
    })

In [19]:
navegador.quit()

In [20]:
# mostra os dados formatados
texto = f'{"Banco Central":<43}{"Taxa Atual":<15}{"Reunião":<18}{"Última Mudança"}\n\n'
for bc in bancos_centrais:
    texto += f'{bc["Banco Central"]:<43}{bc["Taxa Atual"]:<15}{bc["Reunião"]:<18}{bc["Última Mudança"]}\n'

In [21]:
# indicadores :v:
melhor_taxa_atual = max(bancos_centrais, key=lambda banco: formatar(banco))['Banco Central']
pior_taxa_atual = min(bancos_centrais, key=lambda banco: formatar(banco))['Banco Central']

In [22]:
# adicionar ao texto
texto += f'\nBanco com a melhor taxa atual: {melhor_taxa_atual}'
texto += f'\nBanco com a pior taxa atual: {pior_taxa_atual}'

In [23]:
# print(texto)

Banco Central                              Taxa Atual     Reunião           Última Mudança

Banco Central do Brasil (BCB)              13,75%         26.10.2022        03.08.2022 (50bp)
Banco Central Europeu (ECB)                1,25%          27.10.2022        08.09.2022 (75bp)
Reserva Federal (FED)                      3,25%          02.11.2022        21.09.2022 (75bp)
Banco da Inglaterra (BOE)                  2,25%          03.11.2022        22.09.2022 (50bp)
Banco Nacional Suíço (SNB)                 0,50%          15.12.2022        22.09.2022 (75bp)
Banco da Reserva da Austrália (RBA)        2,60%          04.10.2022        04.10.2022 (25bp)
Banco do Canadá (BOC)                      3,25%          26.10.2022        07.09.2022 (75bp)
Banco do Japão (BOJ)                       -0,10%         28.10.2022        29.01.2016 (-20bp)
Banco Central da Federação Russa (CBR)     7,50%          28.10.2022        16.09.2022 (-50bp)
Banco de Reserva da Índia (RBI)            5,90%          07

In [25]:
# criar arquivo
arquivo = open(fr'logs/log {formatar_data(datetime.now())}.txt', 'a')
# escrever no arquivo
arquivo.writelines(texto)
# fechar arquivo
arquivo.close()